In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
base_url = 'https://www.numbeo.com/cost-of-living/historical-data-city-selector'

In [3]:
page = requests.get(base_url)
numbeo_city_soup = BeautifulSoup(page.content, "html.parser")
results = numbeo_city_soup.find('table', class_='related_links')
print(results())

[<tr><td style="width: 20%">
<h4>A</h4>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aachen">Aachen, Germany</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aalborg">Aalborg, Denmark</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aarhus-Denmark">Aarhus, Denmark</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abbotsford">Abbotsford, Canada</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aberdeen">Aberdeen, United Kingdom</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abidjan">Abidjan, Ivory Coast</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abu-Dhabi">Abu Dhabi, United Arab Emirates</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abuja">Abuja, Nigeria</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Accra">Accra, Ghana</a><br/>
<a href="https://www.numbeo.com/cost-of-living/ci

In [4]:
list_cities = results.find_all('a')
list_cities[0]

<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aachen">Aachen, Germany</a>

In [5]:
len(list_cities)

964

In [6]:
city_name = lambda x: f"({x[0]}) {x[1]}" if len(x) > 2 else x[0]

city_dict = lambda x: {'City':city_name(x.text.split(",")), 'Country':x.text.split(",")[-1], 'Url':x["href"]}
city_pages = [city_dict(city) for city in list_cities]

In [7]:
city_pages[0]

{'City': 'Aachen',
 'Country': ' Germany',
 'Url': 'https://www.numbeo.com/cost-of-living/city-history/in/Aachen'}

In [8]:
df = pd.DataFrame(city_pages)
df.head()

,City,Country,Url
0,Aachen,Germany,https://www.numbeo.com/cost-of-living/city-his...
1,Aalborg,Denmark,https://www.numbeo.com/cost-of-living/city-his...
2,Aarhus,Denmark,https://www.numbeo.com/cost-of-living/city-his...
3,Abbotsford,Canada,https://www.numbeo.com/cost-of-living/city-his...
4,Aberdeen,United Kingdom,https://www.numbeo.com/cost-of-living/city-his...


In [9]:
df.shape

(964, 3)

In [10]:
countries = df["Country"].unique()

In [11]:
len(countries)

145

In [12]:
cities = df["City"].unique()

In [13]:
df[df["City"].duplicated(keep=False)]

,City,Country,Url
339,Hamilton,Bermuda,https://www.numbeo.com/cost-of-living/city-his...
340,Hamilton,Canada,https://www.numbeo.com/cost-of-living/city-his...
341,Hamilton,New Zealand,https://www.numbeo.com/cost-of-living/city-his...
369,Hyderabad,India,https://www.numbeo.com/cost-of-living/city-his...
370,Hyderabad,Pakistan,https://www.numbeo.com/cost-of-living/city-his...
427,Kingston,Canada,https://www.numbeo.com/cost-of-living/city-his...
428,Kingston,Jamaica,https://www.numbeo.com/cost-of-living/city-his...
485,London,Canada,https://www.numbeo.com/cost-of-living/city-his...
486,London,United Kingdom,https://www.numbeo.com/cost-of-living/city-his...


In [14]:
df['City'] == 'Abbotsford'

0      False
1      False
2      False
3       True
4      False
       ...  
959    False
960    False
961    False
962    False
963    False
Name: City, Length: 964, dtype: bool

In [15]:
df.loc[df['City'] == 'Abbotsford']

,City,Country,Url
3,Abbotsford,Canada,https://www.numbeo.com/cost-of-living/city-his...


In [16]:
df.loc[df['City'] == 'Abbotsford'].index[0]

3

In [17]:
df.iloc[df.loc[df['City'] == 'Abbotsford'].index[0]]["Url"]

'https://www.numbeo.com/cost-of-living/city-history/in/Abbotsford'

In [18]:
page = requests.get(df.iloc[df.loc[df['City'] == 'Abbotsford'].index[0]]["Url"])

In [19]:
page

<Response [200]>

In [20]:
abbotsford = BeautifulSoup(page.content, "html.parser")

In [21]:
abbotsford

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script defer="" src="https://www.numbeo.com/common/numbeo_standard.js"></script>
<link href="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.10.3/themes/redmond/jquery-ui.min.css" rel="Stylesheet"/>
<link href="https://www.numbeo.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<script defer="" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
<script defer="" src="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.10.3/jquery-ui.min.js"></script>
<script src="https://www.gstatic.com/charts/loader.js"></script>
<link href="https://www.numbeo.com/images/apple-touch-icon.png" rel="apple-touch-icon"/>
<script>
  if (typeof whenDocReady === "function") {
    // already declared, do nothing
  } else {
    function whenDocReady(fn) {
      // see if DOM is already available
      if (document.readyState === "complete" || document.readyS

In [22]:
inner_width = abbotsford.find_all('div', class_='innerWidth')

In [23]:
len(inner_width)

3

In [24]:
inner_width[2]

<div class="innerWidth">
<div class="right-ads-and-info">
<div class="right-ads">
<script async="" src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<!-- Small Right 468x60 -->
<ins class="adsbygoogle" data-ad-client="ca-pub-8993465169534557" data-ad-slot="7246910313" style="display:inline-block;width:468px;height:60px"></ins>
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script>
</div>
</div>
<p>
<nav class="breadcrumb" itemscope="" itemtype="http://schema.org/BreadcrumbList">
<a class="discreet_link" href="https://www.numbeo.com/common/"><svg class="image_without_border hide_smaller_than_600" height="16" style="vertical-align: text-bottom" viewbox="0 0 24 24" width="16" xmlns="http://www.w3.org/2000/svg"><path d="M10 20v-6h4v6h5v-8h3L12 3L2 12h3v8z" fill="#1B3B6F"></path></svg></a><span itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem">
<a class="breadcrumb_link" href="https://www.numbeo.com/cost-of-living/" itemprop=

In [25]:
def get_tables(city):
    page = requests.get(df.iloc[df.loc[df['City'] == city].index[0]]["Url"])
    one_city_soup = BeautifulSoup(page.content, "html.parser")
    inner_width = one_city_soup.find_all('div', class_='innerWidth')
    results = inner_width[2].find_all('table')
    return results

In [26]:
cape_town_data = get_tables("Cape Town")
len(cape_town_data) 

14

In [27]:
cape_town_data[0]

<table class="stripe row-border order-column compact" id="tier_1">
<thead>
<tr>
<th><div class="font_in_table_headers">Year</div></th><th><div class="font_in_table_headers">Meal, Inexpensive Restaurant</div></th><th><div class="font_in_table_headers">Meal for 2 People, <br/>Mid-range Restaurant, Three-course</div></th><th><div class="font_in_table_headers">McMeal at McDonalds <br/>(or Equivalent Combo Meal)</div></th></tr>
</thead>
<tbody>
<tr>
<td style="text-align: right">2023</td>
<td style="white-space: nowrap; text-align: right">170.00</td>
<td style="white-space: nowrap; text-align: right">700.00</td>
<td style="white-space: nowrap; text-align: right">80.00</td>
</tr>
<tr>
<td style="text-align: right">2022</td>
<td style="white-space: nowrap; text-align: right">150.00</td>
<td style="white-space: nowrap; text-align: right">600.00</td>
<td style="white-space: nowrap; text-align: right">70.00</td>
</tr>
<tr>
<td style="text-align: right">2021</td>
<td style="white-space: nowrap; t

In [28]:
nairobi_data = get_tables("Nairobi")
len(nairobi_data)

14

In [30]:
from io import StringIO

In [31]:
reader_converter = lambda x: pd.DataFrame(pd.read_html(StringIO(str(x)))[0])

cape_df_list = [reader_converter(table) for table in cape_town_data]
nairobi_df_list =[reader_converter(table) for table in nairobi_data]

In [69]:
cape_df_list[13]

,Year,1 Pair of Jeans (Levis 501 Or Similar),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes
0,2023,822.46,541.04,1548.28,1367.35
1,2022,824.94,562.93,1545.66,1299.52
2,2021,712.03,533.33,1327.75,1327.16
3,2020,755.44,506.53,1333.99,1293.25
4,2019,788.30,502.96,1382.25,1253.38
5,2018,758.63,456.52,1206.04,1164.58
6,2017,693.21,467.50,1114.10,993.55
7,2016,721.54,501.47,1071.40,1186.17
8,2015,667.00,488.85,943.18,843.52
9,2014,643.25,394.22,924.92,766.58


In [33]:
nairobi_df_list[6]

,Year,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre
0,2023,151500.00,-
1,2022,159800.00,-
2,2021,-,-
3,2020,-,-
4,2019,445622.50,-
5,2018,-,-
6,2017,-,-
7,2016,-,-
8,2015,-,-
9,2014,120000.00,106666.67


In [34]:
import os

def save_df(df_list, city): 
    titles = ['eat-out-meal', 'beverage', 'basic-grocery', 'fresh-produce', 'deli', 'rental', 'property',
              'salary', 'transport-daily', 'transport-monthly', 'mortgage', 'utilities', 'entertainment', 'clothing']
    # Create directory if it doesn't exist
    directory = f'data/{city}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    for index, frame in enumerate(df_list):
        # Ensure the title index is within bounds
        if index < len(titles):
            # Save DataFrame to CSV
            frame.to_csv(f'{directory}/{titles[index]}.csv', sep=',', index=False, encoding='utf-8')


In [35]:
save_df(cape_df_list, 'cape-town')

In [36]:
save_df(nairobi_df_list, 'nairobi')

In [37]:
def markets_df (df_list):
    return pd.concat([df_list[2], df_list[3], df_list[4]], axis=1)

In [38]:
cpt_market = markets_df(cape_df_list)

In [39]:
cpt_market.drop_duplicates()

,Year,"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),Eggs (regular) (12),Water (1.5 liter bottle),Domestic Beer (0.5 liter bottle),Imported Beer (0.33 liter bottle),Year,Apples (1kg),Oranges (1kg),...,"Rice (white), (1kg)",Tomato (1kg),Banana (1kg),Onion (1kg),Year,Local Cheese (1kg),Bottle of Wine (Mid-Range),Cigarettes 20 Pack (Marlboro),Chicken Fillets (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat)
0,2023,18.74,17.11,39.54,16.82,21.19,30.25,2023,26.43,23.56,...,28.66,26.08,28.76,24.90,2023,139.19,85.0,50.0,88.33,120.92
1,2022,16.13,16.66,31.75,14.96,26.14,35.14,2022,25.17,22.55,...,26.18,25.40,24.39,18.28,2022,119.94,80.0,50.0,77.21,111.93
2,2021,14.40,13.74,30.67,16.03,22.13,28.68,2021,22.86,23.72,...,20.88,22.25,22.18,17.27,2021,111.33,80.0,50.0,75.77,113.12
3,2020,14.89,13.88,32.54,13.87,23.67,31.01,2020,24.64,23.57,...,22.65,22.54,23.92,18.86,2020,112.55,80.0,45.0,79.20,108.22
4,2019,16.00,14.00,32.94,18.23,27.69,35.56,2019,25.90,28.71,...,25.40,20.89,23.82,22.67,2019,107.75,80.0,40.0,89.44,114.80
5,2018,12.87,13.14,34.62,14.73,20.95,28.45,2018,21.60,19.50,...,23.00,17.86,17.78,13.58,2018,104.87,65.0,40.0,71.71,102.01
6,2017,12.82,11.97,23.90,14.10,18.33,28.79,2017,18.95,19.00,...,19.00,18.00,17.50,13.39,2017,98.20,60.0,40.0,65.34,92.15
7,2016,13.12,12.44,24.33,13.34,18.72,30.31,2016,20.82,18.43,...,17.15,19.20,16.31,16.40,2016,90.04,60.0,37.5,68.04,78.41
8,2015,11.74,10.87,20.40,13.40,17.53,22.80,2015,16.23,13.23,...,18.57,15.25,13.51,11.03,2015,85.87,50.0,35.0,59.09,95.53
9,2014,11.69,10.35,21.91,13.78,16.30,18.33,2014,16.89,17.36,...,18.45,14.89,-,-,2014,87.00,50.0,34.5,56.50,-


In [40]:
cpt_market.columns


Index(['Year', 'Milk (regular), (1 liter)', 'Loaf of Fresh White Bread (500g)',
       'Eggs (regular) (12)', 'Water (1.5 liter bottle)',
       'Domestic Beer (0.5 liter bottle)', 'Imported Beer (0.33 liter bottle)',
       'Year', 'Apples (1kg)', 'Oranges (1kg)', 'Potato (1kg)',
       'Lettuce (1 head)', 'Rice (white), (1kg)', 'Tomato (1kg)',
       'Banana (1kg)', 'Onion (1kg)', 'Year', 'Local Cheese (1kg)',
       'Bottle of Wine (Mid-Range)', 'Cigarettes 20 Pack (Marlboro)',
       'Chicken Fillets (1kg)',
       'Beef Round (1kg) (or Equivalent Back Leg Red Meat)'],
      dtype='object')

In [57]:
cpt_market.T.drop_duplicates().T


,Year,"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),Eggs (regular) (12),Water (1.5 liter bottle),Domestic Beer (0.5 liter bottle),Imported Beer (0.33 liter bottle),Apples (1kg),Oranges (1kg),Potato (1kg),Lettuce (1 head),"Rice (white), (1kg)",Tomato (1kg),Banana (1kg),Onion (1kg),Local Cheese (1kg),Bottle of Wine (Mid-Range),Cigarettes 20 Pack (Marlboro),Chicken Fillets (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat)
0,2023,18.74,17.11,39.54,16.82,21.19,30.25,26.43,23.56,22.00,18.48,28.66,26.08,28.76,24.90,139.19,85.0,50.0,88.33,120.92
1,2022,16.13,16.66,31.75,14.96,26.14,35.14,25.17,22.55,18.48,17.34,26.18,25.40,24.39,18.28,119.94,80.0,50.0,77.21,111.93
2,2021,14.4,13.74,30.67,16.03,22.13,28.68,22.86,23.72,16.59,15.72,20.88,22.25,22.18,17.27,111.33,80.0,50.0,75.77,113.12
3,2020,14.89,13.88,32.54,13.87,23.67,31.01,24.64,23.57,20.21,15.45,22.65,22.54,23.92,18.86,112.55,80.0,45.0,79.2,108.22
4,2019,16.0,14.0,32.94,18.23,27.69,35.56,25.90,28.71,23.12,18.11,25.40,20.89,23.82,22.67,107.75,80.0,40.0,89.44,114.80
5,2018,12.87,13.14,34.62,14.73,20.95,28.45,21.60,19.50,14.77,13.93,23.00,17.86,17.78,13.58,104.87,65.0,40.0,71.71,102.01
6,2017,12.82,11.97,23.9,14.1,18.33,28.79,18.95,19.00,15.00,13.39,19.00,18.00,17.50,13.39,98.2,60.0,40.0,65.34,92.15
7,2016,13.12,12.44,24.33,13.34,18.72,30.31,20.82,18.43,18.73,14.67,17.15,19.20,16.31,16.40,90.04,60.0,37.5,68.04,78.41
8,2015,11.74,10.87,20.4,13.4,17.53,22.8,16.23,13.23,11.16,10.72,18.57,15.25,13.51,11.03,85.87,50.0,35.0,59.09,95.53
9,2014,11.69,10.35,21.91,13.78,16.3,18.33,16.89,17.36,9.61,9.69,18.45,14.89,-,-,87.0,50.0,34.5,56.5,-


In [52]:
cpt_market

,Year,"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),Eggs (regular) (12),Water (1.5 liter bottle),Domestic Beer (0.5 liter bottle),Imported Beer (0.33 liter bottle),Year,Apples (1kg),Oranges (1kg),...,"Rice (white), (1kg)",Tomato (1kg),Banana (1kg),Onion (1kg),Year,Local Cheese (1kg),Bottle of Wine (Mid-Range),Cigarettes 20 Pack (Marlboro),Chicken Fillets (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat)
0,2023,18.74,17.11,39.54,16.82,21.19,30.25,2023,26.43,23.56,...,28.66,26.08,28.76,24.90,2023,139.19,85.0,50.0,88.33,120.92
1,2022,16.13,16.66,31.75,14.96,26.14,35.14,2022,25.17,22.55,...,26.18,25.40,24.39,18.28,2022,119.94,80.0,50.0,77.21,111.93
2,2021,14.40,13.74,30.67,16.03,22.13,28.68,2021,22.86,23.72,...,20.88,22.25,22.18,17.27,2021,111.33,80.0,50.0,75.77,113.12
3,2020,14.89,13.88,32.54,13.87,23.67,31.01,2020,24.64,23.57,...,22.65,22.54,23.92,18.86,2020,112.55,80.0,45.0,79.20,108.22
4,2019,16.00,14.00,32.94,18.23,27.69,35.56,2019,25.90,28.71,...,25.40,20.89,23.82,22.67,2019,107.75,80.0,40.0,89.44,114.80
5,2018,12.87,13.14,34.62,14.73,20.95,28.45,2018,21.60,19.50,...,23.00,17.86,17.78,13.58,2018,104.87,65.0,40.0,71.71,102.01
6,2017,12.82,11.97,23.90,14.10,18.33,28.79,2017,18.95,19.00,...,19.00,18.00,17.50,13.39,2017,98.20,60.0,40.0,65.34,92.15
7,2016,13.12,12.44,24.33,13.34,18.72,30.31,2016,20.82,18.43,...,17.15,19.20,16.31,16.40,2016,90.04,60.0,37.5,68.04,78.41
8,2015,11.74,10.87,20.40,13.40,17.53,22.80,2015,16.23,13.23,...,18.57,15.25,13.51,11.03,2015,85.87,50.0,35.0,59.09,95.53
9,2014,11.69,10.35,21.91,13.78,16.30,18.33,2014,16.89,17.36,...,18.45,14.89,-,-,2014,87.00,50.0,34.5,56.50,-


In [ ]:
cpt_market.fillna(value=np.nan)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
"Milk (regular), (1 liter)",18.74,16.13,14.4,14.89,16.0,12.87,12.82,13.12,11.74,11.69,11.29,9.32,8.17,8.3
Loaf of Fresh White Bread (500g),17.11,16.66,13.74,13.88,14.0,13.14,11.97,12.44,10.87,10.35,9.09,8.33,7.68,8.38
Eggs (regular) (12),39.54,31.75,30.67,32.54,32.94,34.62,23.9,24.33,20.4,21.91,22.27,17.6,19.51,13.75
Water (1.5 liter bottle),16.82,14.96,16.03,13.87,18.23,14.73,14.1,13.34,13.4,13.78,13.11,11.6,10.95,11.0
Domestic Beer (0.5 liter bottle),21.19,26.14,22.13,23.67,27.69,20.95,18.33,18.72,17.53,16.3,13.58,10.0,10.6,9.0
Imported Beer (0.33 liter bottle),30.25,35.14,28.68,31.01,35.56,28.45,28.79,30.31,22.8,18.33,14.56,11.88,14.46,13.33
Apples (1kg),26.43,25.17,22.86,24.64,25.90,21.60,18.95,20.82,16.23,16.89,13.90,14.20,12.74,-
Oranges (1kg),23.56,22.55,23.72,23.57,28.71,19.50,19.00,18.43,13.23,17.36,11.67,11.83,5.00,-
Potato (1kg),22.00,18.48,16.59,20.21,23.12,14.77,15.00,18.73,11.16,9.61,9.94,9.95,8.48,-


In [ ]:

cpt_market.replace({'-': np.nan}, inplace=True)
cpt_market

,Year,"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),Eggs (regular) (12),Water (1.5 liter bottle),Domestic Beer (0.5 liter bottle),Imported Beer (0.33 liter bottle),Year,Apples (1kg),Oranges (1kg),...,"Rice (white), (1kg)",Tomato (1kg),Banana (1kg),Onion (1kg),Year,Local Cheese (1kg),Bottle of Wine (Mid-Range),Cigarettes 20 Pack (Marlboro),Chicken Fillets (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat)
0,2023,18.74,17.11,39.54,16.82,21.19,30.25,2023,26.43,23.56,...,28.66,26.08,28.76,24.90,2023,139.19,85.0,50.0,88.33,120.92
1,2022,16.13,16.66,31.75,14.96,26.14,35.14,2022,25.17,22.55,...,26.18,25.40,24.39,18.28,2022,119.94,80.0,50.0,77.21,111.93
2,2021,14.40,13.74,30.67,16.03,22.13,28.68,2021,22.86,23.72,...,20.88,22.25,22.18,17.27,2021,111.33,80.0,50.0,75.77,113.12
3,2020,14.89,13.88,32.54,13.87,23.67,31.01,2020,24.64,23.57,...,22.65,22.54,23.92,18.86,2020,112.55,80.0,45.0,79.20,108.22
4,2019,16.00,14.00,32.94,18.23,27.69,35.56,2019,25.90,28.71,...,25.40,20.89,23.82,22.67,2019,107.75,80.0,40.0,89.44,114.80
5,2018,12.87,13.14,34.62,14.73,20.95,28.45,2018,21.60,19.50,...,23.00,17.86,17.78,13.58,2018,104.87,65.0,40.0,71.71,102.01
6,2017,12.82,11.97,23.90,14.10,18.33,28.79,2017,18.95,19.00,...,19.00,18.00,17.50,13.39,2017,98.20,60.0,40.0,65.34,92.15
7,2016,13.12,12.44,24.33,13.34,18.72,30.31,2016,20.82,18.43,...,17.15,19.20,16.31,16.40,2016,90.04,60.0,37.5,68.04,78.41
8,2015,11.74,10.87,20.40,13.40,17.53,22.80,2015,16.23,13.23,...,18.57,15.25,13.51,11.03,2015,85.87,50.0,35.0,59.09,95.53
9,2014,11.69,10.35,21.91,13.78,16.30,18.33,2014,16.89,17.36,...,18.45,14.89,NaN,NaN,2014,87.00,50.0,34.5,56.50,NaN


In [ ]:

def rename_cols(df):
    year_index = "Year"
    col_labels = df.loc[year_index].tolist()
    col_labels = [int(year) for year in col_labels]
    df.columns = col_labels
    df.drop(year_index, inplace=True)
    return df


In [ ]:
new_col_cpt_markets = rename_cols(cpt_market)

In [ ]:
for col in new_col_cpt_markets.columns:
  new_col_cpt_markets[col] = new_col_cpt_markets[col].astype(float)